In [ ]:
#===========================================================================================
# ElasticSearch(이하:ES) 텍스트 임베딩 테스트 예제
#
# => ES 서버는 먼저 구축 후 실행 된 상태이어야 함.
# =>ElasticSearch 7.3.0 버전부터는 cosine similarity 검색을 지원한다.
# =>Index data로 dense_vector 를 집어넣고, 벡터형식의 데이터를 쿼리(검색어)로 하여 코사인 유사도를 측정하여 가장 유사한 데이터를 찾는다.
# => 여기서는 ElasticSearch와 S-BERT를 이용함
# => ElasticSearch에 index 파일은 index.json /데이터 파일은 KorQuAD_v1.0_train_convert.json 참조
#
# => 참고자료 : https://skagh.tistory.com/32
#
# => 참고로 여기서는 title_vector 만 구함, paragrapha_vector는 cpu에서는 엄청 오래 걸려서 주석처리하였음
#===========================================================================================

# sentenceTransformers 라이브러리 설치
#!pip install -U sentence-transfor{

# elasticsearch 서버 접속 모듈 설치
# !pip install elasticsearch

# 한국어 문장 분리기(kss) 설치
#!pip install kss

In [1]:
import torch
from sentence_transformers import SentenceTransformer, util
import kss, numpy
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from tqdm.notebook import tqdm

from elasticsearch import Elasticsearch
from elasticsearch import helpers

# elastic 서버 접속 테스트
#es = Elasticsearch("https://192.168.0.91:9200/", verify_certs=False)
#es = Elasticsearch("http://192.168.0.130:9200/")
#es.info()

device = torch.device('cpu')

In [2]:
# s-bert 모델 테스트
sbert_model_path = 'F:\\AI\\model\\sbert-ts2022-04-01-distiluse-7'

# cpu 모델로 실행할때는 device=cpu, 기본은 GPU임
embedder = SentenceTransformer(sbert_model_path, device=device)

text = '나는 오늘 밥을 먹는다.'
vectors = embedder.encode(text, convert_to_tensor=True)
vector_list = [vector.numpy().tolist() for vector in vectors]

print(f'vector_len:{len(vector_list)}')


vector_len:768


In [3]:
# 인덱싱 함수 
def index_data():
    es.indices.delete(index=INDEX_NAME, ignore=[404])
    
    count = 0
        
    # 인덱스 생성
    with open(INDEX_FILE) as index_file:
        source = index_file.read().strip()
      
        count += 1
        print(f'{count}:{source}')
      
        es.indices.create(index=INDEX_NAME, body=source)
        
    count = 0
    
    # DATA 추기
    with open(DATA_FILE) as data_file:
        for line in data_file:
            line = line.strip()
            
            json_data = json.loads(line)
            docs = []
            
            for j in json_data:
                count += 1
                
                docs.append(j)
                if count % BATCH_SIZE == 0:
                    index_batch(docs)
                    docs = []
                    print("Indexed {} documents.".format(count))
                  
                # ** 500 개만 보냄
                #if count >= 500:
                #    break
                    
            if docs:
                index_batch(docs)
                print("Indexed {} documents.".format(count))
                    
    es.indices.refresh(index=INDEX_NAME)
    print("=== End Done indexing===")
    
    
# 문단(paragraph)들 분리
# 문장으로 나누고, 해당 vector들의 평균을 구함.
# =>굳이 elasticsearch에 문단 벡터는 추가하지 않고, title 벡터만 이용해도 되므로 주석처리함
def paragraph_index(paragraph):
    avg_paragraph_vec = numpy.zeros((1,768))
    sent_count = 0
    
    #print(paragraph)
    # kss로 분할할때 줄바꿈 있으면, 파싱하는데 에러남.따라서 "\n"는 제거함
    paragraph = paragraph.replace("\n","")
    
    #print("==Start paragraph_index==")
    for sent in kss.split_sentences(paragraph):
        # 문장으로 나누고, 해당 vector들의 평균을 구함.
        avg_paragraph_vec += embed_text([sent])
        sent_count += 1
        
        # **cpu로 하면 너무 오래걸리므로, cpu일때는 앞에 1개만 embedding 시킴
        #if sent_count > 1:
        #    break
            
    avg_paragraph_vec /= sent_count
    return avg_paragraph_vec.ravel(order='C')

def index_batch(docs):
    titles = [doc["title"] for doc in docs]
    title_vectors = embed_text(titles)
    
    # * cpu로 문단은 임베딩하는데 너무 오래 걸리므로 주석처리함
    #paragraph_vectors = [paragraph_index(doc["paragraph"]) for doc in tqdm(docs)]
    requests = []
    
    for i, doc in enumerate(tqdm(docs)):
        request = doc
        request["_op_type"] = "index"
        request["_index"] = INDEX_NAME
        request["title_vector"] = title_vectors[i]
        #request["paragraph_vector"] = paragraph_vectors[i]  # * cpu로 문단은 임베딩하는데 너무 오래 걸리므로 주석처리함
        requests.append(request)
        
    # batch 단위로 한꺼번에 es에 데이터 insert 시킴     
    bulk(es, requests)
    
# embedding 모델에서 vector를 구함    
def embed_text(input):
    vectors =  embedder.encode(input, convert_to_tensor=True)
    return [vector.numpy().tolist() for vector in vectors]
          

In [4]:
#======================================================================================
# ElasticSearch(이하:ES) 데이터 인텍싱
# - ElasticSearch(이하:ES)에 KorQuAD_v1.0_train_convert.json 파일에 vector값을 구하여 index 함
#
# => index 명 : korquad
# => index 구조 : index_1.json 파일 참조
# => BATCH_SIZE : 100 => 100개의 vector값을 구하여, 한꺼번에 ES에 인텍스 데이터를 추가함.
#======================================================================================
INDEX_NAME = 'korquad'
INDEX_FILE = './data/index.json'
DATA_FILE = './data/KorQuAD_v1.0_train_convert.json'
BATCH_SIZE = 100

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.130:9200/")
print(es.info())

# 2. index 처리
index_data()

C:\Users\bong9\anaconda3\envs\pytorch\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'lenovo-x240', 'cluster_name': 'mpower', 'cluster_uuid': 'JJ1h3dNTRvSLJFW3gj5MCw', 'version': {'number': '7.17.3', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '5ad023604c8d7416c9eb6c0eadb62b14e766caff', 'build_date': '2022-04-19T08:11:19.070913226Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}
1:{
  "settings": {
    "number_of_shards": 2,
    "number_of_replicas": 1
  },
   "mappings": {
    "dynamic": "true",
    "_source": {
      "enabled": "true"
    },
    "properties": {
      "title": {
        "type": "text"
      },
	  "paragraph": {
        "type": "text"
      },
      "title_vector": {
        "type": "dense_vector",
        "dims": 768
      },
	  "paragraph_vector": {
        "type": "dense_vector",
        "dims": 768
      }
    }
  }
}


  0%|          | 0/100 [00:00<?, ?it/s]

Indexed 100 documents.


  0%|          | 0/100 [00:00<?, ?it/s]

...

Indexed 1300 documents.


  0%|          | 0/100 [00:00<?, ?it/s]

Indexed 1400 documents.


  0%|          | 0/20 [00:00<?, ?it/s]

Indexed 1420 documents.
=== End Done indexing===


In [ ]:
# kibana 콘솔창에 접속해서 계수 확인
# http://192.168.0.130:5601/app/dev_tools 에 접속해서 해야함

# GET korquad/_count

In [19]:
# 검색 하기

import time
from elasticsearch import Elasticsearch

def run_query_loop():
    while True:
        try:
            handle_query()
        except KeyboardInterrupt:
            return
        
def handle_query():
    
    query = input("검색어 입력: ")
    
    start_embedding_time = time.time()
    query_vector = embed_text([query])[0]
    end_embedding_time = time.time() - start_embedding_time
    
    # 쿼리 구성
    script_query = {
        "script_score":{
            "query":{
                "match_all": {}},
            "script":{
                "source": "cosineSimilarity(params.query_vector, doc['title_vector']) + 1.0",  
                "params": {"query_vector": query_vector}
            }
        }
    }
    
    print('query\n')
    print(script_query)
    
    # 실제 ES로 검색 쿼리 날림
    start_search_time = time.time()
    response = es.search(
        index=INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source":{"includes": ["title", "paragraph"]}
        }
    )
    end_search_time = time.time() - start_search_time
    
    print("{} total hits.".format(response["hits"]["total"]["value"])) 
    print("embedding time: {:.2f} ms".format(end_embedding_time * 1000)) 
    print("search time: {:.2f} ms".format(end_search_time * 1000)) 
    
    # 쿼리 응답 결과값에서 _id, _score, _source 등을 뽑아냄
    for hit in response["hits"]["hits"]: 
        print("id: {}, score: {}".format(hit["_id"], hit["_score"])) 
        print(hit["_source"]) 
        print()


In [22]:
#====================================================================
# ES 인덱싱된 내용 검색 
# => cosineSimilarity 스크립트를 이용하여 ES로 query 함(*이때 SEARCH_SIZE를 몇개로 할지 지정할수 있음)
# => 쿼리 응답 결과 값에서 _id, _score, _source 등을 뽑아냄
#====================================================================

INDEX_NAME = 'korquad'
SEARCH_SIZE = 3

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.130:9200/")
print(es.info)
# 2. query 처리
run_query_loop()

<bound method Elasticsearch.info of <Elasticsearch([{'host': '192.168.0.130', 'port': 9200}])>>
검색어 입력: 손홍민이 골을 넣었다
query

{'script_score': {'query': {'match_all': {}}, 'script': {'source': "cosineSimilarity(params.query_vector, doc['title_vector']) + 1.0", 'params': {'query_vector': [-0.0024203795474022627, 0.011293071322143078, 0.018497295677661896, -0.02111748978495598, 0.013731654733419418, 0.020461151376366615, -0.04164306819438934, -0.014276704750955105, -0.029905220493674278, -0.03804846480488777, -0.06705421209335327, 0.004300708882510662, -0.017926668748259544, 0.009397072717547417, 0.005591812077909708, -0.04081070050597191, 0.01940281130373478, -0.01319345086812973, -0.022125737741589546, 0.006233416497707367, -0.03671857342123985, 0.01473719347268343, -0.016591880470514297, 0.032445330172777176, 0.014549210667610168, 0.0035431161522865295, 0.007819305174052715, 0.04061303660273552, -0.03290097415447235, -0.04555494338274002, 0.009110452607274055, 0.029045304283499718, 0.047